# Aliran Kerja Human-in-the-Loop dengan Microsoft Agent Framework

## 🎯 Objektif Pembelajaran

Dalam notebook ini, anda akan belajar cara melaksanakan aliran kerja **human-in-the-loop** menggunakan `RequestInfoExecutor` dari Microsoft Agent Framework. Corak yang berkuasa ini membolehkan anda menghentikan aliran kerja AI untuk mendapatkan input manusia, menjadikan agen anda interaktif dan memberikan manusia kawalan terhadap keputusan penting.

## 🔄 Apa itu Human-in-the-Loop?

**Human-in-the-loop (HITL)** adalah corak reka bentuk di mana agen AI menghentikan pelaksanaan untuk meminta input manusia sebelum meneruskan. Ini penting untuk:

- ✅ **Keputusan kritikal** - Mendapatkan kelulusan manusia sebelum mengambil tindakan penting
- ✅ **Situasi yang tidak jelas** - Membolehkan manusia menjelaskan apabila AI tidak pasti
- ✅ **Keutamaan pengguna** - Meminta pengguna memilih antara beberapa pilihan
- ✅ **Pematuhan & keselamatan** - Memastikan pengawasan manusia untuk operasi yang dikawal selia
- ✅ **Pengalaman interaktif** - Membina agen perbualan yang bertindak balas terhadap input pengguna

## 🏗️ Bagaimana Ia Berfungsi dalam Microsoft Agent Framework

Kerangka ini menyediakan tiga komponen utama untuk HITL:

1. **`RequestInfoExecutor`** - Executor khas yang menghentikan aliran kerja dan mengeluarkan `RequestInfoEvent`
2. **`RequestInfoMessage`** - Kelas asas untuk muatan permintaan yang ditaip yang dihantar kepada manusia
3. **`RequestResponse`** - Mengaitkan respons manusia dengan permintaan asal menggunakan `request_id`

**Corak Aliran Kerja:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Contoh Kita: Tempahan Hotel dengan Pengesahan Pengguna

Kita akan membina aliran kerja bersyarat dengan menambah pengesahan manusia **sebelum** mencadangkan destinasi alternatif:

1. Pengguna meminta destinasi (contohnya, "Paris")
2. `availability_agent` memeriksa jika bilik tersedia
3. **Jika tiada bilik** → `confirmation_agent` bertanya "Adakah anda ingin melihat alternatif?"
4. Aliran kerja **dihentikan** menggunakan `RequestInfoExecutor`
5. **Manusia menjawab** "ya" atau "tidak" melalui input konsol
6. `decision_manager` mengarahkan berdasarkan respons:
   - **Ya** → Menunjukkan destinasi alternatif
   - **Tidak** → Membatalkan permintaan tempahan
7. Paparkan hasil akhir

Ini menunjukkan cara memberikan pengguna kawalan terhadap cadangan agen!

---

Mari kita mulakan! 🚀


## Langkah 1: Import Perpustakaan yang Diperlukan

Kami mengimport komponen standard Agent Framework serta **kelas khusus human-in-the-loop**:
- `RequestInfoExecutor` - Executor yang menghentikan aliran kerja untuk input manusia
- `RequestInfoEvent` - Acara yang dikeluarkan apabila input manusia diminta
- `RequestInfoMessage` - Kelas asas untuk muatan permintaan yang ditaip
- `RequestResponse` - Mengaitkan respons manusia dengan permintaan
- `WorkflowOutputEvent` - Acara untuk mengesan output aliran kerja


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Langkah 2: Tentukan Model Pydantic untuk Output Berstruktur

Model-model ini menentukan **skema** yang akan dikembalikan oleh agen. Kami mengekalkan semua model daripada aliran kerja bersyarat dan menambah:

**Baru untuk Human-in-the-Loop:**
- `HumanFeedbackRequest` - Subkelas `RequestInfoMessage` yang menentukan muatan permintaan yang dihantar kepada manusia
  - Mengandungi `prompt` (soalan untuk ditanya) dan `destination` (konteks tentang bandar yang tidak tersedia)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Langkah 3: Cipta Alat Tempahan Hotel

Alat yang sama dari aliran kerja bersyarat - memeriksa sama ada bilik tersedia di destinasi.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Langkah 4: Tentukan Fungsi Keadaan untuk Penghalaan

Kita memerlukan **empat fungsi keadaan** untuk aliran kerja human-in-the-loop kita:

**Daripada aliran kerja bersyarat:**
1. `has_availability_condition` - Menghala apabila hotel ADA tersedia
2. `no_availability_condition` - Menghala apabila hotel TIDAK tersedia

**Baru untuk human-in-the-loop:**
3. `user_wants_alternatives_condition` - Menghala apabila pengguna berkata "ya" kepada alternatif
4. `user_declines_alternatives_condition` - Menghala apabila pengguna berkata "tidak" kepada alternatif


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Langkah 5: Cipta Pelaksana Pengurus Keputusan

Ini adalah **teras corak manusia-dalam-gelung**! `DecisionManager` ialah `Executor` tersuai yang:

1. **Menerima maklum balas manusia** melalui objek `RequestResponse`
2. **Memproses keputusan pengguna** (ya/tidak)
3. **Menghalakan aliran kerja** dengan menghantar mesej kepada ejen yang sesuai

Ciri utama:
- Menggunakan penghias `@handler` untuk mendedahkan kaedah sebagai langkah aliran kerja
- Menerima `RequestResponse[HumanFeedbackRequest, str]` yang mengandungi permintaan asal dan jawapan pengguna
- Menghasilkan mesej ringkas "ya" atau "tidak" yang mencetuskan fungsi syarat kami


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Langkah 6: Cipta Pelaksana Paparan Tersuai

Pelaksana paparan yang sama dari aliran kerja bersyarat - menghasilkan keputusan akhir sebagai output aliran kerja.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Langkah 7: Muatkan Pembolehubah Persekitaran

Konfigurasikan klien LLM (GitHub Models, Azure OpenAI, atau OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Langkah 8: Cipta Ejen AI dan Pelaksana

Kami mencipta **enam komponen aliran kerja**:

**Ejen (dibungkus dalam AgentExecutor):**
1. **availability_agent** - Memeriksa ketersediaan hotel menggunakan alat
2. **confirmation_agent** - 🆕 Menyediakan permintaan pengesahan manusia
3. **alternative_agent** - Mencadangkan bandar alternatif (apabila pengguna berkata ya)
4. **booking_agent** - Menggalakkan tempahan (apabila bilik tersedia)
5. **cancellation_agent** - 🆕 Menguruskan mesej pembatalan (apabila pengguna berkata tidak)

**Pelaksana Khas:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` yang menghentikan aliran kerja untuk input manusia
7. **decision_manager** - 🆕 Pelaksana khas yang mengarahkan berdasarkan respons manusia (sudah ditakrifkan di atas)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Langkah 9: Bina Aliran Kerja dengan Manusia dalam Gelung

Sekarang kita bina graf aliran kerja dengan **penghalaan bersyarat** termasuk laluan manusia dalam gelung:

**Struktur Aliran Kerja:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Tepi Utama:**
- `availability_agent → confirmation_agent` (apabila tiada bilik)
- `confirmation_agent → prepare_human_request` (ubah jenis)
- `prepare_human_request → request_info_executor` (jeda untuk manusia)
- `request_info_executor → decision_manager` (sentiasa - menyediakan RequestResponse)
- `decision_manager → alternative_agent` (apabila pengguna berkata "ya")
- `decision_manager → cancellation_agent` (apabila pengguna berkata "tidak")
- `availability_agent → booking_agent` (apabila bilik tersedia)
- Semua laluan berakhir di `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Langkah 10: Jalankan Kes Ujian 1 - Bandar TANPA Ketersediaan (Paris dengan Pengesahan Manusia)

Ujian ini menunjukkan **kitaran penuh manusia-dalam-gelung**:

1. Meminta hotel di Paris
2. availability_agent memeriksa → Tiada bilik
3. confirmation_agent mencipta soalan untuk manusia
4. request_info_executor **memberhentikan aliran kerja** dan mengeluarkan `RequestInfoEvent`
5. **Aplikasi mengesan acara dan meminta pengguna di konsol**
6. Pengguna menaip "yes" atau "no"
7. Aplikasi menghantar respons melalui `send_responses_streaming()`
8. decision_manager mengarahkan berdasarkan respons
9. Hasil akhir dipaparkan

**Corak Utama:**
- Gunakan `workflow.run_stream()` untuk iterasi pertama
- Gunakan `workflow.send_responses_streaming(pending_responses)` untuk iterasi berikutnya
- Dengar `RequestInfoEvent` untuk mengesan bila input manusia diperlukan
- Dengar `WorkflowOutputEvent` untuk menangkap hasil akhir


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Langkah 11: Jalankan Kes Ujian 2 - Bandar DENGAN Ketersediaan (Stockholm - Tiada Input Manusia Diperlukan)

Ujian ini menunjukkan **laluan langsung** apabila bilik tersedia:

1. Minta hotel di Stockholm  
2. availability_agent memeriksa → Bilik tersedia ✅  
3. booking_agent mencadangkan tempahan  
4. display_result menunjukkan pengesahan  
5. **Tiada input manusia diperlukan!**

Aliran kerja sepenuhnya mengelakkan laluan manusia-dalam-gelung apabila bilik tersedia.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Poin Penting dan Amalan Terbaik Human-in-the-Loop

### ✅ Apa yang Anda Pelajari:

#### 1. **Corak RequestInfoExecutor**
Corak human-in-the-loop dalam Microsoft Agent Framework menggunakan tiga komponen utama:
- `RequestInfoExecutor` - Memberhentikan aliran kerja dan mengeluarkan acara
- `RequestInfoMessage` - Kelas asas untuk muatan permintaan yang ditaip (subkelas ini!)
- `RequestResponse` - Mengaitkan respons manusia dengan permintaan asal

**Pemahaman Kritikal:**
- `RequestInfoExecutor` TIDAK mengumpulkan input sendiri - ia hanya memberhentikan aliran kerja
- Kod aplikasi anda mesti mendengar `RequestInfoEvent` dan mengumpulkan input
- Anda mesti memanggil `send_responses_streaming()` dengan kamus yang memetakan `request_id` kepada jawapan pengguna

#### 2. **Corak Pelaksanaan Penstriman**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Seni Bina Berasaskan Acara**
Dengar acara tertentu untuk mengawal aliran kerja:
- `RequestInfoEvent` - Input manusia diperlukan (aliran kerja diberhentikan)
- `WorkflowOutputEvent` - Hasil akhir tersedia (aliran kerja selesai)
- `WorkflowStatusEvent` - Perubahan keadaan (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, dll.)

#### 4. **Executor Tersuai dengan @handler**
`DecisionManager` menunjukkan cara untuk mencipta executor yang:
- Menggunakan penghias `@handler` untuk mendedahkan kaedah sebagai langkah aliran kerja
- Menerima mesej yang ditaip (contohnya, `RequestResponse[HumanFeedbackRequest, str]`)
- Mengarahkan aliran kerja dengan menghantar mesej kepada executor lain
- Mengakses konteks melalui `WorkflowContext`

#### 5. **Penghalaan Bersyarat dengan Keputusan Manusia**
Anda boleh mencipta fungsi syarat yang menilai respons manusia:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Aplikasi Dunia Sebenar:

1. **Aliran Kerja Kelulusan**
   - Mendapatkan kelulusan pengurus sebelum memproses laporan perbelanjaan
   - Memerlukan semakan manusia sebelum menghantar e-mel automatik
   - Mengesahkan transaksi bernilai tinggi sebelum pelaksanaan

2. **Moderasi Kandungan**
   - Menandakan kandungan yang meragukan untuk semakan manusia
   - Meminta moderator membuat keputusan akhir pada kes tepi
   - Meningkatkan kepada manusia apabila keyakinan AI rendah

3. **Perkhidmatan Pelanggan**
   - Membiarkan AI mengendalikan soalan rutin secara automatik
   - Meningkatkan isu kompleks kepada ejen manusia
   - Bertanya kepada pelanggan jika mereka ingin bercakap dengan manusia

4. **Pemprosesan Data**
   - Meminta manusia menyelesaikan entri data yang tidak jelas
   - Mengesahkan tafsiran AI terhadap dokumen yang tidak jelas
   - Membiarkan pengguna memilih antara beberapa tafsiran yang sah

5. **Sistem Kritikal Keselamatan**
   - Memerlukan pengesahan manusia sebelum tindakan yang tidak boleh diubah
   - Mendapatkan kelulusan sebelum mengakses data sensitif
   - Mengesahkan keputusan dalam industri yang dikawal selia (kesihatan, kewangan)

6. **Ejen Interaktif**
   - Membina bot perbualan yang bertanya soalan susulan
   - Mencipta wizard yang membimbing pengguna melalui proses kompleks
   - Merancang ejen yang bekerjasama dengan manusia langkah demi langkah

### 🔄 Perbandingan: Dengan vs Tanpa Human-in-the-Loop

| Ciri | Aliran Kerja Bersyarat | Aliran Kerja Human-in-the-Loop |
|------|------------------------|-------------------------------|
| **Pelaksanaan** | Tunggal `workflow.run()` | Gelung dengan `run_stream()` + `send_responses_streaming()` |
| **Input Pengguna** | Tiada (sepenuhnya automatik) | Prompt interaktif melalui `input()` atau UI |
| **Komponen** | Ejen + Executor | + RequestInfoExecutor + DecisionManager |
| **Acara** | Hanya AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, dll. |
| **Berhenti** | Tiada berhenti | Aliran kerja berhenti di RequestInfoExecutor |
| **Kawalan Manusia** | Tiada kawalan manusia | Manusia membuat keputusan utama |
| **Kes Penggunaan** | Automasi | Kerjasama & pengawasan |

### 🚀 Corak Lanjutan:

#### Pelbagai Titik Keputusan Manusia
Anda boleh mempunyai beberapa nod `RequestInfoExecutor` dalam aliran kerja yang sama:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Pengendalian Timeout
Melaksanakan timeout untuk respons manusia:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integrasi UI Kaya
Daripada `input()`, integrasi dengan UI web, Slack, Teams, dll.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Human-in-the-Loop Bersyarat
Hanya meminta input manusia dalam situasi tertentu:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Amalan Terbaik:

1. **Sentiasa Subkelas RequestInfoMessage**
   - Memberikan keselamatan jenis dan pengesahan
   - Membolehkan konteks kaya untuk rendering UI
   - Menjelaskan niat setiap jenis permintaan

2. **Gunakan Prompt Deskriptif**
   - Sertakan konteks tentang apa yang anda tanya
   - Jelaskan akibat setiap pilihan
   - Pastikan soalan ringkas dan jelas

3. **Tangani Input Tidak Dijangka**
   - Mengesahkan respons pengguna
   - Memberikan lalai untuk input yang tidak sah
   - Memberikan mesej ralat yang jelas

4. **Jejak ID Permintaan**
   - Gunakan korelasi antara request_id dan respons
   - Jangan cuba menguruskan keadaan secara manual

5. **Reka Bentuk untuk Tidak Menyekat**
   - Jangan menyekat thread menunggu input
   - Gunakan corak async sepanjang masa
   - Sokong contoh aliran kerja serentak

### 📚 Konsep Berkaitan:

- **Agent Middleware** - Memintas panggilan ejen (notebook sebelumnya)
- **Pengurusan Keadaan Aliran Kerja** - Mengekalkan keadaan aliran kerja antara pelaksanaan
- **Kerjasama Multi-Ejen** - Menggabungkan human-in-the-loop dengan pasukan ejen
- **Seni Bina Berasaskan Acara** - Membina sistem reaktif dengan acara

---

### 🎓 Tahniah!

Anda telah menguasai aliran kerja human-in-the-loop dengan Microsoft Agent Framework! Anda kini tahu cara:
- ✅ Memberhentikan aliran kerja untuk mengumpulkan input manusia
- ✅ Menggunakan RequestInfoExecutor dan RequestInfoMessage
- ✅ Mengendalikan pelaksanaan penstriman dengan acara
- ✅ Mencipta executor tersuai dengan @handler
- ✅ Mengarahkan aliran kerja berdasarkan keputusan manusia
- ✅ Membina ejen AI interaktif yang bekerjasama dengan manusia

**Ini adalah corak kritikal untuk membina sistem AI yang boleh dipercayai dan terkawal!** 🚀



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
